In [92]:
# importing essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [93]:
# importing dataset
data=pd.read_csv('/content/drive/MyDrive/BTC/btc.csv')
data.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
# duplicate data as df
df=data.copy()

In [95]:
# creating date column
df['date'] = pd.to_datetime(df['Timestamp'],unit='s').dt.date
df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,date
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39,2011-12-31
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31


In [97]:
# grouping data
group = df.groupby('date')
real_price = group['Weighted_Price'].mean()

In [98]:
real_price

date
2011-12-31        4.471603
2012-01-01        4.806667
2012-01-02        5.000000
2012-01-03        5.252500
2012-01-04        5.208159
                  ...     
2021-03-27    55193.357260
2021-03-28    55832.958824
2021-03-29    56913.993819
2021-03-30    58346.912268
2021-03-31    58764.349363
Name: Weighted_Price, Length: 3376, dtype: float64

In [10]:
# setting train and test data
prediction_days = 30
train= real_price[:len(real_price)-prediction_days]
test= real_price[len(real_price)-prediction_days:]

In [99]:
# scaling the data
training_set = train.values
training_set = np.reshape(training_set, (len(training_set), 1))
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
training_set = sc.fit_transform(training_set)

In [100]:
# initialising x_train and y_train
X_train = training_set[0:len(training_set)-1]
y_train = training_set[1:len(training_set)]

In [101]:
# reshaping in 3-D because our layers need like this
X_train = np.reshape(X_train, (len(X_train), 1,1))

In [102]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# Initialising the RNN
regressor = Sequential()

# Adding the input layer and the LSTM layer
regressor.add(LSTM(units = 10, activation = 'relu', input_shape = (None, 1)))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, batch_size = 5, epochs = 100)

Epoch 1/100
669/669 [==============================] - 3s 3ms/step - loss: 0.0091
Epoch 2/100
669/669 [==============================] - 2s 3ms/step - loss: 0.0016
Epoch 3/100
669/669 [==============================] - 2s 3ms/step - loss: 9.8913e-05
Epoch 4/100
669/669 [==============================] - 2s 3ms/step - loss: 7.8241e-05
Epoch 5/100
669/669 [==============================] - 2s 3ms/step - loss: 6.5992e-05
Epoch 6/100
669/669 [==============================] - 2s 3ms/step - loss: 5.3717e-05
Epoch 7/100
669/669 [==============================] - 2s 3ms/step - loss: 4.8093e-05
Epoch 8/100
669/669 [==============================] - 2s 3ms/step - loss: 4.2991e-05
Epoch 9/100
669/669 [==============================] - 2s 4ms/step - loss: 4.3529e-05
Epoch 10/100
669/669 [==============================] - 2s 4ms/step - loss: 4.0520e-05
Epoch 11/100
669/669 [==============================] - 2s 3ms/step - loss: 4.0611e-05
Epoch 12/100
174/669 [======>.......................] - ETA:

KeyboardInterrupt: ignored

In [103]:
test[:10]

,date,Weighted_Price
0,2021-03-02,48617.289321
1,2021-03-03,50429.449016
2,2021-03-04,49398.250399
3,2021-03-05,47717.643300
4,2021-03-06,48371.543538
5,2021-03-07,50218.216804
6,2021-03-08,50798.354212
7,2021-03-09,53980.847262
8,2021-03-10,55375.366241
9,2021-03-11,56272.596972


In [27]:
# Making the predictions
test_set = test.values
inputs = np.reshape(test_set, (len(test_set), 1))
inputs = sc.transform(inputs)
inputs = np.reshape(inputs, (len(inputs), 1, 1))
predicted_BTC_price = regressor.predict(inputs)


In [28]:
# converting to normal numbers
predicted_BTC_price = sc.inverse_transform(predicted_BTC_price)

In [45]:
test=test.drop(['index'],axis=1)

In [79]:
predicted_BTC_price.shape

(30, 1)

In [80]:
pred=pd.DataFrame(predicted_BTC_price)

In [63]:
pred.head()

,0
0,48773.562500
1,50502.437500
2,49518.812500
3,47914.710938
4,48539.003906
5,50300.996094
6,50854.199219
7,53885.984375
8,55212.804688
9,56065.902344


In [86]:
final=test.copy()
final=pd.merge(final,pred,on=final.index)
final=final.drop(['key_0'],axis=1)
final = final.rename(columns={0: 'predicted'})
final

,date,Weighted_Price,predicted
0,2021-03-02,48617.289321,48773.562500
1,2021-03-03,50429.449016,50502.437500
2,2021-03-04,49398.250399,49518.812500
3,2021-03-05,47717.643300,47914.710938
4,2021-03-06,48371.543538,48539.003906
5,2021-03-07,50218.216804,50300.996094
6,2021-03-08,50798.354212,50854.199219
7,2021-03-09,53980.847262,53885.984375
8,2021-03-10,55375.366241,55212.804688
9,2021-03-11,56272.596972,56065.902344


In [91]:
import plotly.express as px
import plotly.graph_objects as go

fig=px.line(final,x='date',y='predicted')
fig.add_scatter(x=final['date'], y=final['Weighted_Price'])
fig.show()
